In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2     # actually reload imports
#%config InlineBackend.figure_format = 'svg'    # use vectorial figures
from profile.profil import *
from smartg import RoughSurface, LambSurface, FlatSurface
from smartgx import smartg, smartg_thr, reptran_merge
from water.iop_spm import IOP_SPM
from water.iop_mm import IOP_MM
from luts import MLUT, LUT, Idx, merge, read_lut_hdf, read_mlut_hdf
from PAR import Int, Int2, Irr, ReadREPTRAN_bands, dailyPAR, viewPAR, ObsPAR, simulatePAR
import pickle

In [ ]:
#fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=True,verbose=True)
# store file lists
#with open('simulations_list_cirrus-continental.pickle4', 'w') as f:
 # pickle.dump([fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l], f)
# recompute date list
#fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=False,verbose=True)
# Getting back the file lists:
with open('simulations_list_cirrus-continental.pickle4') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
    #--------------------------
    # spectral and spatial sampling
    #--------------------------
    LMIN=380. # nm
    LMAX=2220. # nm
    PFRES=100. # spectral resolution for phase function (nm)
    SAMPLING = 1 # Reptran band undersampling rate
    grid='100[30]10[5]10[1]0' # altitude grid for scattering and absorption coefficients
    pfgrid=[100, 20, 10, 8., 2., 0.] # altitude grid for phase functions
    pfwav=np.linspace(LMIN, LMAX, num=(LMAX-LMIN)/PFRES+1, endpoint=True) # wavelength grid for phase functions
    repname='reptran_solar_coarse' # k distribution 
    #repname='reptran_solar_sentinel'
    rep = REPTRAN(repname+'.cdf')
    o3=300. # DU
    wref=550. # wavelength for AOT and COT reference
    aot=0.265
    #atmmodel, wvc = ('afglms', 2.96) # g/cm2
    #atmmodel, wvc = ('afglt', 4.18) # g/cm2
    atmmodel, wvc, SurfPre = ('afglmw', 0.89, 1018.) # g/cm2, mbar
    #atmmodel, wvc = ('afglsw', 0.42) # g/cm2
    #atmmodel, wvc = ('afglss', 2.11) # g/cm2
       
    
    aermodel_l=['maritime_clean','continental_average','maritime_polluted']
    aermodel=aermodel_l[0]
    cloudscatt='wc.sol.mie' # scattering properties of water cloud in the solar spectrum through Mie calculations
    cloudname='Stratocumulus'
    #cloudname='Cirrus'
    wc = 1 # water cloud droplets arbitrary concentration
    cot = 1. 
    reff = 12. # cloud droplets effective radius (mic)
    Zcmin = 9. # cloud bottom height (km)
    Zcmax = 10. # cloud top height (km)
    chl=0.1 # mg/l
    ws=5. # m/s
    lat=45.
    lon=-40.
    da_l=['2015/7/15','2015/1/15'] # date of acquisition
    dir='/home/did/RTC/SMART-G/tools/PAR/VIIRS4/'
#    dir='/tmp/'
    vza_l=[0., 30., 60.]
    raa=90. # Sun VIIRS relative azimuth
    time_step=120. # time step for daily PAR integration (minute)
    
    wi,wb,we,ex,dl,ibands = ReadREPTRAN_bands(repname,LMIN=LMIN,LMAX=LMAX,SAMPLING=SAMPLING,FULL=True)
    aer = AeroOPAC(aermodel, aot, wref)
    pro = Profile(atmmodel,
    grid=grid,  # optional, otherwise use default grid
    pfgrid=pfgrid,   # optional, otherwise use a single band 100-0
    pfwav=pfwav # optional, otherwise phase functions are calculated at all bands
    ,aer=aer
    #,cloud=CloudOPAC(cloudname,[(cloudscatt, wc, reff , Zcmin, Zcmax)], cot, wref)
    )
    pro.calc(665.)            
    aer.calc(665.)
    t665=aer.tau_tot
    aer.calc(865.)
    t865=aer.tau_tot
    Angstrom = -np.log(t665/t865)/np.log(665./865.)

 #   Angstrom=0.
    
    surf=RoughSurface(SUR=3, WIND=ws, NH2O=1.34)
    #water=IOP_SPM(SPM=100.,pfwav=pfwav,NANG=7201)
    water=IOP_MM(chl,pfwav=pfwav,NANG=7201)
    filename = tempfile.mktemp(dir='/tmp/', prefix='fpp_')
    smartg(wl = ibands, THVDEG=55.,
    atm=pro, NBPHOTONS=1e8,OUTPUT_LAYERS=3,
    surf=surf, NFAER=10000,NFOCE=100000).save(filename, compress=False)
  #  surf=surf, water=water, NFAER=10000,NFOCE=100000).save(filename, compress=False)

In [ ]:
#f=filename
f='/tmp/fpp_NRQGee'
E, Q = viewPAR(f, f,verbose=True)

In [ ]:
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=False,verbose=True)
with open('simulations_list_cirrus-continental.pickle4') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=True)

In [ ]:
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l=simulatePAR(CALC=False)

In [ ]:
# Getting back the objects:
with open('simulations_list_cirrus-continental.pickle4') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l,verbose=False)

In [ ]:
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=False,verbose=True)
with open('simulations_list_continental.pickle4') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=True)

In [ ]:
with open('simulations_list_stratocumulus5.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l, verbose=True)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=True)

In [ ]:
with open('simulations_list_cirrus.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=False)

In [ ]:
with open('simulations_list_stratocumulus50.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l,verbose=False)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=False)